In [ ]:
# import packages
from tensorflow import keras
from keras.models import load_model
import cozmo
from cozmo.world import EvtNewCameraImage
import numpy
import time
from cozmo.util import ImageBox, Rotation, degrees, speed_mmps, distance_mm

# Cozmo can see or not: camera is on (True) or off (False)
camera = False

# Cozmo is searching for a sign or not 
searching = False

# Cozmo has a prediction or not
sign_id = False

# The sign label (0,1,2,3) and its probability
sign = 0
proba = 0

# path to save model
save_model_path ='\\Model\\'

# path to new images
path_new_img = '\\NewImg\\'


# load the model and the model weights
model = keras.models.load_model(save_model_path+'MyModel.h5')
model.load_weights(save_model_path+'model_weights.h5')

# generator to rescale new images
data_gen = ImageDataGenerator(rescale=1./255)

# Reset Cozmo's body to a neutral position
def tuck_your_shirt_cozmo(robot: cozmo.robot.Robot):
    ''' lift must be down. lift can be 32 to 92mm above the ground'''
    robot.set_lift_height(0.0).wait_for_completed()
    ''' head should look straight ahead. Cozmo looks down at -25 and looks up at 44.5'''
    robot.set_head_angle(degrees(13.0)).wait_for_completed()

# Cozmo samples new images
def new_image(evt, **kwargs):
    global camera
    global sign_id
    global searching
    global sign
    global proba
    ''' Cozmo's backpack lights are blue to shoiw it is active'''
    robot.set_all_backpack_lights(cozmo.lights.blue_light)
    if camera:
        if not searching:
            if not sign_id:
                searching = True
                pilImage = kwargs['image'].raw_image
                global imagename
                pilImage.save(path_new_img+f"\\{imagename}-{kwargs['image'].image_number}.jpeg", "JPEG")
                sign = model.predict(data_gen.flow_from_directory(path_new_img))
                proba = model.predict_proba(model, data_gen.flow_from_directory(path_new_img), verbose=0)
                sign_id = True
                searching = False

# Core program
def cozmo_thinker_finder(robot: cozmo.robot.Robot):
    global camera
    global sign
    global proba
    global sign_id
    label = np.argmax(sign[0])
    ''' Cozmo gets ready'''
    tuck_your_shirt_cozmo(robot)
    ''' This event handler gets Cozmo interested '''
    robot.add_event_handler(cozmo.world.EvtNewCameraImage, new_image)
    ''' Cozmo tries to find a known letter, if not rotates 30 degrees to the next'''
    while not sign_id:
            camera = False
            robot.turn_in_place(degrees(-30)).wait_for_completed()
            camera = True
    camera = False
    ''' Cozmo has a prediction with a probability'''
    if sign_id:
        if proba>0.8:
            if label==0:
                robot.say_text("This is the letter C like Cozmo").wait_for_completed()
            elif label==1:
                robot.say_text("This is the letter O").wait_for_completed()
            elif label==2:
                robot.say_text("This is the letter Z").wait_for_completed()
            elif label==3:
                robot.say_text("This is the letter M").wait_for_completed()
        else:
            robot.say_text("I don't know this one").wait_for_completed()
     ''' turn off backpack lights when done'''
    robot.set_all_backpack_lights(cozmo.lights.off_light)
    time.sleep(2)

# Run core program
cozmo.run_program(cozmo_thinker_finder, use_viewer = True, force_viewer_on_top = True)